# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [111]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.890326411333684, 4.440583029700601, 6.329544736039495, 5.959293661606207, 5.811097544855288, 5.041350461493732, 4.800822031316854, 5.207161731091848, 5.396634154803546, 4.9395054095724635]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [112]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[5.816779507878609, 9.56729220197961, 8.3192413619661, 8.789541895092105, 7.936728224890414, 8.577939397531871, 7.149042671331649, 8.10065257418663, 7.835032251674348, 9.455739161232463]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [113]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.9395054095724635, 9.455739161232463, 5.396634154803546, 5.207161731091848, 7.835032251674348, 4.800822031316854, 5.041350461493732, 5.811097544855288, 8.10065257418663, 5.959293661606207]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

the means, variances, and probablity of each distribution

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [116]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[9.455739161232463, 7.835032251674348, 8.10065257418663]
[4.9395054095724635, 5.396634154803546, 5.207161731091848, 4.800822031316854, 5.041350461493732, 5.811097544855288, 5.959293661606207]
P(S_1) = 0.3,  P(S_2) = 0.7
mean_1 = 8.463807995697813,  mean_2 = 5.307980713534278
var_1 = 0.5037227445347187,  var_2 = 0.16563352457653174


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [118]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.9395054095724635
probability of observing that point if it came from cluster 0 =  1.858243502849937e-11
probability of observing that point if it came from cluster 1 =  0.20281175445690378
point =  9.455739161232463
probability of observing that point if it came from cluster 0 =  0.11393981617303055
probability of observing that point if it came from cluster 1 =  1.6245227079284508e-136
point =  5.396634154803546
probability of observing that point if it came from cluster 0 =  7.042920659799715e-09
probability of observing that point if it came from cluster 1 =  2.0871488514827887
point =  5.207161731091848
probability of observing that point if it came from cluster 0 =  6.642702039488689e-10
probability of observing that point if it came from cluster 1 =  2.0012842813978673
point =  7.835032251674348
probability of observing that point if it came from cluster 0 =  0.36338849657252875
probability of observing that point if it came from cluster 1 =  6.852530105098928e-51
poin

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [94]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [
    sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
    sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)
]
print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

#these values are more specific and closer to the actual

P(S_1) = 1.7465696419341758e-53,  P(S_2) = 1.0
mean_1 = 5.914157506595271,  mean_2 = 6.164150083628297
var_1 = 3.700893426507142,  var_2 = 3.7417066416142157


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [101]:

prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_i = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x_i)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x_i)

    # Update prob_s based on the current point
    prob_s[0] = sum(prob_s0_x) / len(prob_s0_x)
    prob_s[1] = sum(prob_s1_x) / len(prob_s1_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

# Recompute overall probabilities for each cluster after the iterations
prob_s = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [
    sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
    sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)
]
var = [
    sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
    sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)
]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))




# the values are getting closer to the actual


point =  8.51538171867618
point =  7.964258733538877
point =  3.5227296158237067
point =  5.82367016201469
point =  8.547699745469314
point =  6.357148391481212
point =  4.021458416074763
point =  3.6716389853885456
point =  8.131535970551521
point =  5.085979097264154
8.51538171867618
Probability of coming from S_1 = 1.6804240481185027e-53
Probability of coming from S_2 = 1.0

7.964258733538877
Probability of coming from S_1 = 1.6360911474539575e-53
Probability of coming from S_2 = 1.0

3.5227296158237067
Probability of coming from S_1 = 1.7456710482718852e-53
Probability of coming from S_2 = 1.0

5.82367016201469
Probability of coming from S_1 = 1.7125696436667612e-53
Probability of coming from S_2 = 1.0

8.547699745469314
Probability of coming from S_1 = 1.6283877517853226e-53
Probability of coming from S_2 = 1.0

6.357148391481212
Probability of coming from S_1 = 1.6892792382227634e-53
Probability of coming from S_2 = 1.0

4.021458416074763
Probability of coming from S_1 = 1.758012

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [119]:


probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p)

    # Find the maximum probability between p[1] and p[2]
    if p[1] > p[2]:
        print('cluster 1')
    else:
        print('cluster 2')
    


(4.9395054095724635, 3.926745147248991e-11, 0.9999999999607325)
cluster 2
(9.455739161232463, 1.0, 3.3268027915807766e-135)
cluster 1
(5.396634154803546, 1.4461807848424741e-09, 0.9999999985538193)
cluster 2
(5.207161731091848, 1.4225226914012125e-10, 0.9999999998577478)
cluster 2
(7.835032251674348, 1.0, 4.400039369079534e-50)
cluster 1
(4.800822031316854, 5.037210487139433e-11, 0.999999999949628)
cluster 2
(5.041350461493732, 4.870065704428502e-11, 0.9999999999512994)
cluster 2
(5.811097544855288, 1.350087528138711e-05, 0.9999864991247187)
cluster 2
(8.10065257418663, 1.0, 1.712732797946008e-61)
cluster 1
(5.959293661606207, 0.0013739663623557263, 0.9986260336376444)
cluster 2
